Install dependencies

In [5]:
!pip install mysql-connector-python pandas scikit-learn ydata-profiling matplotlib seaborn

In [6]:
import pandas as pd
import mysql.connector
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error, r2_score
from ydata_profiling import ProfileReport
import seaborn as sns
import matplotlib.pyplot as plt

MySQL Database Connection

In [7]:
db_config = {
    'host': 'localhost',
    'user': 'root',
    'password': 'root',
    'database': 'PKK65QSteam'
}
conn = mysql.connector.connect(**db_config)

dim_game = pd.read_sql("SELECT * FROM dim_game", conn)
fact_metrics = pd.read_sql("SELECT * FROM fact_game_metrics", conn)

df = pd.merge(dim_game, fact_metrics, on='AppID')

C:\Users\gyoni\AppData\Local\Temp\ipykernel_2840\1426311818.py:9: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  dim_game = pd.read_sql("SELECT * FROM dim_game", conn)
C:\Users\gyoni\AppData\Local\Temp\ipykernel_2840\1426311818.py:10: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  fact_metrics = pd.read_sql("SELECT * FROM fact_game_metrics", conn)


Drop cells that can't be used for prediction

In [ ]:
drop_columns = [
    'Peak_CCU', 'Positive', 'Negative',
    'Recommendations', 'steamspy_positive', 'steamspy_negative',
    'Header_image', 'Estimated_owners'
]
df.drop(columns=drop_columns, inplace=True)
df = df[df['Estimated_Revenue'].notnull()]

Convert boolean OS support columns to integers

In [ ]:
df['Windows'] = df['Windows'].astype(int)
df['Mac'] = df['Mac'].astype(int)
df['Linux'] = df['Linux'].astype(int)

Convert list columns

In [ ]:
for col in ['Genres', 'Tags', 'Categories']:
    df[col] = df[col].fillna('').apply(lambda x: [i.strip() for i in x.split(',')])
    mlb = df[col].explode().str.get_dummies().groupby(level=0).sum()
    df = df.join(mlb).drop(columns=[col])

Run profiling

In [ ]:
profile = ProfileReport(df, title="Game Dataset Profiling Report", explorative=True)
profile.to_notebook_iframe()